<a href="https://colab.research.google.com/github/rosslogan702/expected_goals_model/blob/master/first_xg_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Building an xG (Expected Goals) Model**

The following notebook walks through building an expected goals model from scratch using a dataset provided by SciSports.   

The dataset is an artificial dataset however this is enough to walk through the process of attempting to build an xG model from scratch.

#### **What is xG?**

The best definition I can find of xG after a quick google search is the following from the official Bundesliga website which states that:

*   Expected goals (xG) is a predictive model used to asess every goal-scoring chance and give a prediction on the likelihood of scoring
*   An xG model computes for each chance the probability to score based on a number of features that we know about the chance e.g distance from goal, was the shot taken by a foot or a head
*  The higher the xG, the higher the probability of scoring. With 1 being the maximum value assigned to a shot and 0 being the minimum.

The predictive model that assigns a probability of scoring to each chance is what we are going to create in this notebook.

[Bundesliga definition](https://www.bundesliga.com/en/bundesliga/news/expected-goals-xg-model-what-is-it-and-why-is-it-useful-sportec-solutions-3177)

### **Download Dataset**

Download the initial dataset.

In [46]:
!wget "https://github.com/JanVanHaaren/how-to-expected-goals/raw/master/shots.parquet" -O "shots.parquet"

--2020-04-19 15:42:36--  https://github.com/JanVanHaaren/how-to-expected-goals/raw/master/shots.parquet
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JanVanHaaren/how-to-expected-goals/master/shots.parquet [following]
--2020-04-19 15:42:37--  https://raw.githubusercontent.com/JanVanHaaren/how-to-expected-goals/master/shots.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7793234 (7.4M) [application/octet-stream]
Saving to: ‘shots.parquet’

shots.parquet       100%[===================>]   7.43M  17.8MB/s    in 0.4s    

2020-04-19 15:42:37 (17.8 MB/s) - ‘shots.parquet’ saved [7793234/7793234]

### **Import Libraries**

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### **View Dataset**

View the dataset downloaded and have a look at some of the features in the dataset.

In [48]:
shots_df = pd.read_parquet('shots.parquet')
print('Number of shots in dataset: {}'.format(shots_df.shape[0]))

Number of shots in dataset: 127643


In [49]:
shots_df.head(5)

,action_game_id,action_team_id,action_player_id,action_period,action_seconds,action_type_id,action_type_name,action_body_part_id,action_result,action_start_x,action_start_y,action_end_x,action_end_y,action1_game_id,action1_team_id,action1_player_id,action1_period,action1_seconds,action1_type_id,action1_type_name,action1_body_part_id,action1_result,action1_start_x,action1_start_y,action1_end_x,action1_end_y,action2_game_id,action2_team_id,action2_player_id,action2_period,action2_seconds,action2_type_id,action2_type_name,action2_body_part_id,action2_result,action2_start_x,action2_start_y,action2_end_x,action2_end_y
0,0,6,5703,1,188.23,11,shot,0,0,98.75,40.17,105.0,40.85,0.0,6.0,3726.0,1.0,187.75,1.0,cross,0.0,1.0,99.28,49.54,99.28,40.70,0.0,6.0,4070.0,1.0,185.62,0.0,pass,0.0,1.0,93.53,65.36,98.78,49.04
1,0,6,3726,1,609.31,11,shot,0,0,84.20,48.48,105.0,27.40,0.0,6.0,5797.0,1.0,607.57,0.0,pass,0.0,1.0,97.48,60.35,83.83,48.11,0.0,6.0,3726.0,1.0,606.29,0.0,pass,0.0,1.0,89.66,61.61,98.06,60.93
2,0,281,2690,1,969.28,11,shot,0,0,92.68,46.52,105.0,41.08,0.0,281.0,6886.0,1.0,967.00,0.0,pass,0.0,1.0,88.35,62.71,92.55,46.39,0.0,281.0,2690.0,1.0,963.64,0.0,pass,0.0,1.0,72.30,60.37,88.05,62.41
3,0,6,4070,1,1193.44,11,shot,0,0,94.40,46.14,104.9,37.30,0.0,6.0,3726.0,1.0,1191.37,0.0,pass,1.0,1.0,89.30,64.65,94.55,46.29,0.0,6.0,5748.0,1.0,1189.19,0.0,pass,0.0,1.0,73.04,15.18,88.79,64.14
4,0,6,6514,1,1359.36,11,shot,1,0,93.52,27.95,105.0,40.87,0.0,6.0,6514.0,1.0,1357.96,21.0,dribble,0.0,1.0,96.79,28.07,93.64,28.07,0.0,6.0,1015.0,1.0,1356.12,3.0,freekick_crossed,0.0,1.0,79.67,6.67,96.47,27.75


In [50]:
print("Features included in the dataset are: \n{}".format(shots_df.columns))

Features included in the dataset are: 
Index(['action_game_id', 'action_team_id', 'action_player_id', 'action_period',
       'action_seconds', 'action_type_id', 'action_type_name',
       'action_body_part_id', 'action_result', 'action_start_x',
       'action_start_y', 'action_end_x', 'action_end_y', 'action1_game_id',
       'action1_team_id', 'action1_player_id', 'action1_period',
       'action1_seconds', 'action1_type_id', 'action1_type_name',
       'action1_body_part_id', 'action1_result', 'action1_start_x',
       'action1_start_y', 'action1_end_x', 'action1_end_y', 'action2_game_id',
       'action2_team_id', 'action2_player_id', 'action2_period',
       'action2_seconds', 'action2_type_id', 'action2_type_name',
       'action2_body_part_id', 'action2_result', 'action2_start_x',
       'action2_start_y', 'action2_end_x', 'action2_end_y'],
      dtype='object')


There are a number of features in the dataset which are all explained below.   

The definitions of the features come from https://github.com/JanVanHaaren/how-to-expected-goals/blob/master/how-to-expected-goals.ipynb



### **Feature Definitions**

For each shot, the dataset contains the following information for the shot as well as the two actions immediately preceding the shot:



*   game_id: a unique identifier of the game;
*   team_id: a unique identifier of the team who performed the action;
*   player_id: a unique identifier of the player who performed the action;
*   period: 1 for the first half and 2 for the second half;
*   seconds: the time elapsed in seconds since the start of the half;
*   type_id: the identifier for the type of action;
*   type_name: the name for the type of action;
*   body_part_id: 0 for foot, 1 for head, 2 for other body part;
*   result: the result of the action: 0 for failure, 1 for success;
*   start_x: the x coordinate for the location where the action started, ranges from 0 to 105;
*   start_y: the y coordinate for the location where the action started, ranges from 0 to 68;
*   end_x: the x coordinate for the location where the action ended, ranges from 0 to 105;
*   end_y: the y coordinate for the location where the action ended, ranges from 0 to 68.

The prefix action refers to the shot, whereas the prefixes action1 and action2 refer to the last and one-but-last action prior to the shot.

The mapping between the type_id and type_name values is as follows:

*   0: pass
*   1: cross
*   2: throw in
*   3: freekick crossed
*   4: freekick short
*   5: corner crossed
*   6: corner short
*   7: take on
*   8: foul
*   9: tackle
*  10: interception
*  11: shot
*  12: shot penalty
*  13: shot freekick
*  14: keeper save
*  18: clearance
*  21: dribble
*  22: goalkick

#### **Create a Baseline Model**

Firstly, we are going to create a very basic and standard baseline model for predicting xG values for shots.  

This model is going to be extremely basic and will contain a limited feature set without doing anything complicated.  

The purpose of creating such a model is that it can serve as a barometer for more complicated models further down the line. That is, more complicated models are only worth if they outperform the simple model.

The very first model in fact that we will create will be "dumb" in a sense as we will just create a python method that always returns 1. That is, it will always predict that the shot taken was scored.

In [51]:
# Sanity check to ensure that we are dealing with all shots
shots_df.action_type_name.unique()

array(['shot'], dtype=object)

In [52]:
# Of the shots, how many ended up in a success (indicated by 1, ie a goal) and how many ended up in a failure state (indicated by 0)
shots_df.action_result.value_counts()

0    114102
1     13541
Name: action_result, dtype: int64

We can see from the above query that there are more shots that result in failure stats than success states in the dataset. This is OK but we will need to be careful when splitting the data into a training & testing set.

#####**Train/Test Split**

Split the data into a training & test set to enable us to evaluate the model performance fairly.  

In [53]:
shot_features_df = shots_df.drop(columns=['action_result'])
shot_features_df.shape

(127643, 38)

In [0]:
# X_ variables are the features and y_ are the results
X_train, X_test, y_train, y_test = train_test_split(shot_features_df, 
                                                    shots_df['action_result'],
                                                    train_size=0.8, 
                                                    stratify=shots_df['action_result'],
                                                    random_state=42)

In [55]:
# Let's check how many successes (shots that led to a goal) & failures (shots that did not lead to a goal) are in the dataset
print('y_train: {}'.format(y_train.value_counts()))
# Let's check the same for y_test
print('y_test: {}'.format(y_test.value_counts()))

y_train: 0    91281
1    10833
Name: action_result, dtype: int64
y_test: 0    22821
1     2708
Name: action_result, dtype: int64


#### **Baseline Predictions**

As we now have our first train/test split, we are now going to calculate our first set of predictions.  

The very first model that we are going to use is going to be a dumb model that just always returns 1 (success or a goal) without taking into account any features.  

This as we mentioned above, is to provide a baseline for more complicated models in the future to be compared against.



In [0]:
def xg_predict(feature_set):
  # Figure out the number of rows
  num_rows = feature_set.shape[0]
  # Return numpy array of 1's for required num rows
  return np.ones((num_rows), dtype=int)

In [58]:
y_train_preds = xg_predict(X_train)
unique_elements, counts_elements = np.unique(y_train_preds, return_counts=True)
print("Frequency of unique values of y_train predictions:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of y_train predictions:
[[     1]
 [102114]]


In [59]:
y_test_preds = xg_predict(X_test)
unique_elements, counts_elements = np.unique(y_test_preds, return_counts=True)
print("Frequency of unique values of y_test predictions:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of y_test predictions:
[[    1]
 [25529]]


As expected, the function makes a prediction of 1 for all shots. Now that we have this we need a way of evaluating the performance of this "dumb" model.